In [1]:
import argparse
import os
import sys
sys.path.append('C:/Users/arive/1 pipeline')
import matplotlib.pyplot as plt
import cv2
from PIL import Image,ImageDraw

In [2]:
from config.options import Options

from data.dataset import OCRDatasetInstanceSeg

In [3]:
cfgfile = "C:/Users/arive/config/config.ini"
opts =  Options(cfgfile)

In [4]:
opts.region_classes

{'paragraph': 0,
 'header': 1,
 'heading': 2,
 'footnote': 3,
 'marginalia': 4,
 'page-number': 5,
 'catch-word': 6,
 'signature-mark': 7,
 'TOC-entry': 8,
 'separator': 9,
 'decoration': 10,
 'figure': 11,
 'other': 12,
 'unknown': 13}

In [5]:
dataset_files = [
    ["C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved", 
     "C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved/page"]]
dataset = OCRDatasetInstanceSeg(dataset_files, opts.region_classes)

In [9]:
dataset_files = [
    ["C:/Users/arive/Datasets/TextDetection/1762_v1_text_improved", 
     "C:/Users/arive/Datasets/TextDetection/1762_v1_text_improved/page"]]
dataset = OCRDatasetInstanceSeg(dataset_files, opts.region_classes)

In [9]:
dataset.__len__()

446

In [10]:
file_path = "C:/Users/arive/Datasets/1762_v1_text_improved.txt"
with open(file_path, "w", encoding="utf-8") as f:
    for i in range(dataset.__len__()):
        page = dataset.__getXMLitem__(i)
        f.write("Page: " + str(i+1) + "\n")
        for reg in page:
            f.write("Region: " + reg['type'] + "\n")
            for line in reg['lines']:
            
                f.write("Line:" + line['TextEquiv'].decode('utf-8') + "\n")        

f.close()

In [11]:
def XML2text(opts, infolder, inpage, textfile, elements, fieldname=False):
    dataset_files = [[infolder, os.path.join(infolder, inpage)]]
    dataset = OCRDatasetInstanceSeg(dataset_files, opts.region_classes)
    if not os.path.exists(textfile):
        os.makedirs(textfile)
    with open(textfile, "w", encoding="utf-8") as f:
        for idx in range(dataset.__len__()):
            page = dataset.__getXMLitem__(idx)
            if fieldname:
                f.write("Page: " + str(idx+1) + "\n")
            
            for i, reg in enumerate(page):
                if reg['type'] in elements:
                    if fieldname:
                        f.write("Region "+str(i+1)+":"  + reg['type'] + "\n")
                    for j, line in enumerate(reg['lines']):
                        if fieldname:
                            f.write("Line "+str(j+1)+":" + line['TextEquiv'].decode('utf-8') + "\n")     
                        else:
                            f.write(line['TextEquiv'].decode('utf-8') + "\n")        

    f.close()

In [10]:
infolder = "C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved"
inpage = "page"
file_path = "C:/Users/arive/Datasets/1778_1-2_Anglo-Saxons_content.txt"

infolder = "C:/Users/arive/Datasets/TextDetection/1762_v1_text_improved" 
inpage = "page"
file_path = "C:/Users/arive/Datasets/1762_v1_text_content.txt"

elements = ['paragraph', 'heading']
#XML2text(opts, infolder, inpage, file_path, elements)

In [13]:
page = dataset.__getXMLitem__(10)

In [14]:
page

[{'id': 'heading_0',
  'type': 'heading',
  'polygon': array([[1436,  584],
         [1568,  585],
         [1667,  586],
         [1703,  588],
         [1718,  589],
         [1726,  590],
         [1732,  591],
         [1742,  593],
         [1751,  595],
         [1759,  597],
         [1766,  599],
         [1772,  601],
         [1778,  604],
         [1781,  606],
         [1784,  609],
         [1786,  612],
         [1788,  616],
         [1790,  622],
         [1791,  628],
         [1792,  638],
         [1792,  690],
         [1791,  695],
         [1789,  701],
         [1787,  704],
         [1780,  711],
         [1770,  716],
         [1758,  719],
         [1748,  721],
         [1727,  725],
         [1720,  726],
         [1705,  727],
         [1607,  728],
         [1474,  728],
         [ 574,  722],
         [ 464,  721],
         [ 452,  720],
         [ 442,  719],
         [ 433,  718],
         [ 419,  716],
         [ 414,  715],
         [ 406,  713],
    

In [6]:
def XML2words(opts, infolder, inpage, textfile, elements, fieldname=False):
    printedIdx = 61
    rcolor = (255, 0, 0)  # RGB color (red in this case)
    bcolor = (0, 0, 0)
    mapping = {}
    dataset_files = [[infolder, os.path.join(infolder, inpage)]]
    dataset = OCRDatasetInstanceSeg(dataset_files, opts.region_classes)
    text_dir = os.path.dirname(textfile)
    if not os.path.exists(text_dir):
        os.makedirs(text_dir)
    total_len = 0
    all_lines = ""
    l_bracket = '{'
    r_bracket = '}'
    with open(textfile, "w", encoding="utf-8") as f:
        for idx in range(dataset.__len__()):
            if idx == printedIdx:
                image = Image.open(dataset.__getfullname__(idx))
                draw = ImageDraw.Draw(image)
            page = dataset.__getXMLitem__(idx)
            if fieldname:
                f.write("Page: " + str(idx+1) + "\n")
            
            for i, reg in enumerate(page):
                if reg['type'] in elements:
                    if fieldname:
                        f.write("Region "+str(i+1)+":"  + reg['type'] + "\n")
                    for j, line in enumerate(reg['lines']):
                        if fieldname:
                            f.write("Line "+str(j+1)+":" + line['TextEquiv'].decode('utf-8') + "\n")     
                        else:
                            line_text = line['TextEquiv'].decode('utf-8') + " "
                            f.write(line_text)
                            line_len = len(line_text)
                            all_lines += line_text
                            line_bbox = line['bbox']
                            line_bbox_len = line_bbox[1][0] - line_bbox[0][0] + 1
                            words = line_text.split(' ')
                            x_base = line_bbox[0][0]
                            x_offset = 0
                            y_base = line_bbox[0][1]
                            width_base = 0
                            height_base = line_bbox[1][1] - line_bbox[0][1] + 1
                            char_offset = 0
                                                                      
                            if line_len > 0:
                                char_len = (line_bbox_len / line_len)
                                for w in words:
                                    if len(w) != 0:
                                        left_x = int(char_offset * char_len)
                                        right_x = int((char_offset + len(w)+1) * char_len )
                                        mapping[total_len+char_offset] = {"page": idx + 1,
                                                                          "x": x_base+left_x,
                                                                          "y": y_base,
                                                                          "width": int((len(w)+1)*char_len),
                                                                          "height": height_base}  
                                        if idx == printedIdx:
                                            top_left = (x_base+left_x, y_base) 
                                            bottom_right = (x_base+right_x, y_base+height_base) 
                                            draw.rectangle([top_left, bottom_right], outline=rcolor, width=3)                                            
                                            
#                                        print("{} {}page:{}, x:{}, y:{}, width:{}, height:{}{}".format(total_len+char_offset,l_bracket, idx + 1, x_base+x_offset,y_base+y_offset,width_base+width_offset,height_base + height_offset,r_bracket))   
                                            print(w, total_len+char_offset, str(mapping[total_len+char_offset]))
                                        char_offset += len(w) + 1 
                                    else:
                                        char_offset += 1
                                    
                            previous_x_offset = x_offset
                            total_len += line_len
            if idx == printedIdx:
                image.show()
    f.close()
    return mapping

In [11]:
infolder = "C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved"
inpage = "page"
file_path = "C:/Users/arive/Datasets/1778_1-2_Anglo-Saxons_content.txt"

elements = ['paragraph', 'heading']
#elements = [ 'footnote']
mapping = XML2words(opts, infolder, inpage, file_path, elements)
mappingfile = "C:/Users/arive/Datasets/1778_1-2_Anglo-Saxons_word_mappings.txt"
with open(mappingfile, "w", encoding="utf-8") as f:
    for item in mapping.keys():
        f.write(str(item)+ " : " + str(mapping[item])+ "\n")
f.close()

abdicating 108628 {'page': 62, 'x': 468, 'y': 403, 'width': 203, 'height': 44}
their 108639 {'page': 62, 'x': 671, 'y': 403, 'width': 110, 'height': 44}
crowns, 108645 {'page': 62, 'x': 782, 'y': 403, 'width': 147, 'height': 44}
sought 108653 {'page': 62, 'x': 929, 'y': 403, 'width': 129, 'height': 44}
for 108660 {'page': 62, 'x': 1059, 'y': 403, 'width': 73, 'height': 44}
a 108664 {'page': 62, 'x': 1133, 'y': 403, 'width': 36, 'height': 44}
secure 108666 {'page': 62, 'x': 1170, 'y': 403, 'width': 129, 'height': 44}
pa 108673 {'page': 62, 'x': 1299, 'y': 403, 'width': 55, 'height': 44}
sheaven 108676 {'page': 62, 'x': 467, 'y': 458, 'width': 151, 'height': 51}
at 108684 {'page': 62, 'x': 618, 'y': 458, 'width': 56, 'height': 51}
the 108687 {'page': 62, 'x': 675, 'y': 458, 'width': 75, 'height': 51}
feet 108691 {'page': 62, 'x': 751, 'y': 458, 'width': 94, 'height': 51}
of 108696 {'page': 62, 'x': 846, 'y': 458, 'width': 56, 'height': 51}
the 108699 {'page': 62, 'x': 903, 'y': 458, 'wid

In [12]:
infolder = "C:/Users/arive/Datasets/TextDetection/1762_v1_text_improved" 
inpage = "page"
file_path = "C:/Users/arive/Datasets/1762_v1_text_content.txt"

elements = ['paragraph', 'heading']
#elements = [ 'footnote']
mapping = XML2words(opts, infolder, inpage, file_path, elements)
mappingfile = "C:/Users/arive/Datasets/1762_v1_text_word_mappings.txt"
with open(mappingfile, "w", encoding="utf-8") as f:
    for item in mapping.keys():
        f.write(str(item)+ " : " + str(mapping[item])+ "\n")
f.close()

ETHELWOTLE. 119315 {'page': 62, 'x': 945, 'y': 501, 'width': 1034, 'height': 89}
HIIS 119327 {'page': 62, 'x': 653, 'y': 635, 'width': 103, 'height': 52}
prince 119332 {'page': 62, 'x': 756, 'y': 635, 'width': 144, 'height': 52}
had 119339 {'page': 62, 'x': 900, 'y': 635, 'width': 82, 'height': 52}
neither 119343 {'page': 62, 'x': 982, 'y': 635, 'width': 164, 'height': 52}
the 119351 {'page': 62, 'x': 1147, 'y': 635, 'width': 82, 'height': 52}
abilities 119355 {'page': 62, 'x': 1230, 'y': 635, 'width': 206, 'height': 52}
nor 119365 {'page': 62, 'x': 1436, 'y': 635, 'width': 82, 'height': 52}
vigour 119369 {'page': 62, 'x': 1518, 'y': 635, 'width': 144, 'height': 52}
of 119376 {'page': 62, 'x': 1662, 'y': 635, 'width': 61, 'height': 52}
his 119379 {'page': 62, 'x': 1724, 'y': 635, 'width': 82, 'height': 52}
father; 119383 {'page': 62, 'x': 1807, 'y': 635, 'width': 164, 'height': 52}
and 119391 {'page': 62, 'x': 1971, 'y': 635, 'width': 82, 'height': 52}
was 119395 {'page': 62, 'x': 2054

In [13]:
len(mapping)

182978

In [93]:
for idx in range(dataset.__len__()):
    print(dataset.__getfullname__(idx))

C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0037.jpg
C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0038.jpg
C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0039.jpg
C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0040.jpg
C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0041.jpg
C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0042.jpg
C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved\rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0043.jpg
C:/Users/arive/Datasets/Tex

In [117]:
image = Image.open("C:/Users/arive/Datasets/TextDetection/1778_1-2_Anglo-Saxons_text_improved/rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0043.jpg")

# Plot the image

draw = ImageDraw.Draw(image)

# Define the coordinates and color of the rectangle
# Here, we're drawing a red rectangle with a thickness of 2 pixels
top_left = (100, 100)  # (x, y) coordinates of the top-left corner
bottom_right = (300, 300)  # (x, y) coordinates of the bottom-right corner
color = (255, 0, 0)  # RGB color (red in this case)

# Draw the rectangle on the image
draw.rectangle([top_left, bottom_right], outline=color, width=2)

# Show the image with the rectangle
image.show()

In [106]:
image